## BART MNLI

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np

In [27]:
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

In [28]:
df = pd.read_csv('cleaned_nli.csv')

In [32]:
premises = df['premise'].tolist()
hypothesis = df['hypothesis'].tolist()
labels = df['labels_encoded'].tolist()

In [59]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
nli_model.to(device)
predictions = []
for i in range(len(premises)):
    encoded_pair = tokenizer.encode(premises[i], hypothesis[i], return_tensors='pt')
    logits = nli_model(encoded_pair.to(device))[0]
    entail_contradiction_logits = logits[:,[0,2]]
    probs = entail_contradiction_logits.softmax(dim=1)
    predicted = torch.argmax(probs)
    predictions.append(predicted.item())

In [60]:
y_pred = np.array(predictions)

In [61]:
y_true = np.array(labels)

In [62]:
target_names = ['entailment','contradiction']

In [63]:
print(classification_report(y_true, y_pred, target_names=target_names))

               precision    recall  f1-score   support

   entailment       0.40      0.26      0.31       896
contradiction       0.45      0.62      0.52       896

     accuracy                           0.44      1792
    macro avg       0.43      0.44      0.42      1792
 weighted avg       0.43      0.44      0.42      1792



## XLM R